In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale

In [4]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")
# sample = df.iloc[:1000, :].copy()
# sample.to_pickle("sample.pkl")

In [5]:
df = pd.read_pickle("donors.pkl")
sample = pd.read_pickle("sample.pkl")

# EDA DONE

In [6]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = sample.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(sample[mailOrderOffers])

for col in groupMailOffers:
    sample[col] = sample[col].fillna(0).astype(np.int16)

In [7]:
searchName = "MDMAUD"
namesToGroup = [col for col in sample.columns.to_list() if searchName in col]
print(namesToGroup)

['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


In [8]:
for name in namesToGroup:
    setattr(sample, name, sample[name].str.replace("X", "").replace("", np.nan))

In [9]:
# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
sample.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [10]:
# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
sample[namesToGroup]

,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G
2,False,NaN,NaN,NaN,NaN,L4E,L,4,E
3,False,NaN,NaN,NaN,NaN,L4E,L,4,E
4,False,NaN,NaN,NaN,NaN,L2F,L,2,F
...,...,...,...,...,...,...,...,...,...
995,False,NaN,NaN,NaN,NaN,L2F,L,2,F
996,False,NaN,NaN,NaN,NaN,L1G,L,1,G
997,False,NaN,NaN,NaN,NaN,L1F,L,1,F
998,False,NaN,NaN,NaN,NaN,L4D,L,4,D


In [11]:
for c in namesToGroup:
    sample[c] = pd.Categorical(sample[c])

In [12]:
sample[namesToGroup].dtypes

MAJOR       category
MDMAUD      category
MDMAUD_R    category
MDMAUD_F    category
MDMAUD_A    category
RFA_2       category
RFA_2R      category
RFA_2F      category
RFA_2A      category
dtype: object

In [13]:
toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col)+"_"+str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: sample.columns.get_loc(col) for col in reorderedCols}

temp = sample[dictIdx.keys()].copy()
sample.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    sample[key] = temp[key]

In [15]:
sample[sample.GEOCODE2.isnull() == True]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,RDATE_22,RAMNT_22,ADATE_23,RFA_23,RDATE_23,RAMNT_23,ADATE_24,RFA_24,RDATE_24,RAMNT_24
577,2006-01-01,BHG,2,FL,33756,,,1990-08-01,0,,...,2014-09-01,9.0,2014-07-01,A3D,NaN,NaN,2014-06-01,A3D,2014-06-01,9.0


In [16]:
# Converting date columns into datetimes
for c in (dateNames := [c for c in sample.columns.values if ("DATE" in c)]):
    sample[c] = pd.to_datetime(sample[c], infer_datetime_format=True)

In [17]:
# Always keep this cell last before EDA WIP
sample.to_pickle("sample_eda_done.pkl")

# EDA WIP

In [18]:
# restart work from here
sample = pd.read_pickle("sample_eda_done.pkl")

In [19]:
sample.dtypes

ODATEDW     datetime64[ns]
OSOURCE             object
TCODE                int64
STATE               object
ZIP                 object
                 ...      
RAMNT_23           float64
ADATE_24    datetime64[ns]
RFA_24              object
RDATE_24    datetime64[ns]
RAMNT_24           float64
Length: 475, dtype: object

In [78]:
df.GEOCODE.unique()

array([' ', '02', '03', '05', '04', '14', '01', '12'], dtype=object)